# 🚀 Hallucination Detection Model Training - Colab (OPTIMIZED)

**DeBERTa-v3-BASE ile hızlı training!**

- **Model:** microsoft/deberta-v3-base (86M params) 🚀
- **GPU:** A100 (Colab Pro) veya T4 ⚡
- **Toplam Süre:** ~3-4 saat (T4), ~3-4 saat (T4)
  - Dataset download: ~5 dakika
  - Dataset conversion: ~2-3 dakika
  - Training: ~1-1.5 saat (A100) veya ~2-3 saat (T4)
- **Output:** Google Drive'a otomatik kaydedilir

---

## ⚙️ KULLANIM
1. **Runtime → Change runtime type → GPU → A100 (veya T4)**
2. **Runtime → Run all** (Ctrl+F9)
3. Google Drive izni ver
4. Bekle ☕ (~1-2 saat A100 için)
5. Model: **Google Drive → MyDrive → hallucination_model_deberta_base/**

---

## 📊 DATASET INFO

### Raw Datasets:
- **WiC**: 6K examples
- **AmbigQA**: 12K examples
- **ASQA**: 5K examples
- **CLAMBER**: 3K examples
- **CondAmbigQA**: 2K examples

### After NLI Conversion (NO AUGMENTATION):
- **Total**: ~28K NLI examples (multiplier=1, orijinal boyut)
- **Expected batches/epoch**: ~875 (batch_size=16)
- **Total batches (3 epochs)**: ~2,625
- **Optimizer steps (grad_accum=2)**: ~1,312

---

## ⚡ OPTIMIZATIONS

### Model:
- **DeBERTa-v3-BASE** (86M params - 5x smaller!)
- **Faster training**: ~1-2 hours (vs 11 hours)
- **Less memory**: 560MB model (vs 1.6GB)

### Configuration:
- **Batch size**: 16 (larger for smaller model)
- **Gradient accumulation**: 4 (effective batch = 64)
- **Epochs**: 3 (faster iteration)
- **Sequence length**: 512 (full)
- **Mixed precision**: FP16

### Performance Optimizations:
- ✓ Pre-tokenized cache
- ✓ DataLoader num_workers=4
- ✓ Prefetch factor=4
- ✓ Drop last incomplete batch
- ✓ Progress bar shows optimizer steps

### Expected Performance:
- **Speed**: 2-3 it/s (faster than large model!)
- **Time**: ~1-2 hours (A100)
- **Memory**: ~10-15GB

---

## 🚀 READY TO RUN!

Optimize edilmiş bu notebook'u çalıştırmak için sadece **Run All** yapın! 🎉

In [ ]:
# ============================================================
# CELL 1: SETUP & GPU CHECK
# ============================================================
print("="*60)
print("🔧 SETUP BAŞLIYOR")
print("="*60)

# GPU kontrol
import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  NO GPU! Runtime → Change runtime type → GPU")

# Mount Google Drive
print("\n📁 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted!")

In [ ]:
# ============================================================
# CELL 2: CLONE CODE & INSTALL DEPENDENCIES
# ============================================================
print("\n" + "="*60)
print("📦 INSTALLING DEPENDENCIES")
print("="*60)

# Clone repository
print("\n[1/3] Cloning code...")
!cd /content && rm -rf rag-project 2>/dev/null || true
!git clone https://github.com/Talha-Dmr/rag-project.git /content/rag-project -q
print("✅ Code cloned")

# Install dependencies
print("\n[2/3] Installing packages...")
!pip install -q pyyaml transformers datasets evaluate scikit-learn accelerate
print("✅ Packages installed")

# Add to path
print("\n[3/3] Setting up path...")
import sys
sys.path.insert(0, '/content/rag-project')
print("✅ Path configured")

In [ ]:
# ============================================================
# CELL 2.5: DOWNLOAD AMBIGUITY DATASETS
# ============================================================
print("\n" + "="*60)
print("📥 DOWNLOADING 5 AMBIGUITY DATASETS")
print("="*60)
print("\n⏱️  This will take 3-5 minutes...\n")

import os
from pathlib import Path
import json

# Create base directory
base_dir = Path('/content/rag-project/data/ambiguity_datasets')
base_dir.mkdir(parents=True, exist_ok=True)

# ============== 1. WiC Dataset ==============
print("[1/5] Downloading WiC dataset...")
wic_dir = base_dir / '01_wic'
wic_dir.mkdir(exist_ok=True)

!wget -q https://pilehvar.github.io/wic/package/WiC_dataset.zip -O /tmp/wic.zip
!unzip -q /tmp/wic.zip -d /tmp/wic_extract
!cp -r /tmp/wic_extract/train {str(wic_dir)}/
!cp -r /tmp/wic_extract/dev {str(wic_dir)}/
!rm -rf /tmp/wic.zip /tmp/wic_extract

print("✅ WiC: ~6K examples")

# ============== 2. AmbigQA Dataset ==============
print("\n[2/5] Downloading AmbigQA dataset...")
ambigqa_dir = base_dir / '02_ambigqa'
ambigqa_dir.mkdir(exist_ok=True)

!wget -q https://nlp.cs.washington.edu/ambigqa/data/train_light.json -O {str(ambigqa_dir)}/train_light.json
!wget -q https://nlp.cs.washington.edu/ambigqa/data/dev_light.json -O {str(ambigqa_dir)}/dev_light.json

print("✅ AmbigQA: ~12K examples")

# ============== 3. ASQA Dataset ==============
print("\n[3/5] Downloading ASQA dataset...")
from datasets import load_dataset

asqa_dir = base_dir / '03_asqa' / 'dataset'
asqa_dir.mkdir(parents=True, exist_ok=True)

asqa_ds = load_dataset('din0s/asqa', split='train')
# Save as JSON with train/dev splits
train_data = [ex for ex in asqa_ds.select(range(int(len(asqa_ds)*0.82)))]
dev_data = [ex for ex in asqa_ds.select(range(int(len(asqa_ds)*0.82), len(asqa_ds)))]

asqa_combined = {'train': train_data, 'dev': dev_data}
with open(asqa_dir / 'ASQA.json', 'w') as f:
    json.dump(asqa_combined, f)

print("✅ ASQA: ~5K examples")

# ============== 4. CLAMBER Dataset ==============
print("\n[4/5] Downloading CLAMBER dataset...")
clamber_dir = base_dir / '04_clamber'
clamber_dir.mkdir(exist_ok=True)

# Download from GitHub raw URL
!wget -q https://raw.githubusercontent.com/yuchenlin/clamber/main/clamber_benchmark.jsonl -O {str(clamber_dir)}/clamber_benchmark.jsonl

print("✅ CLAMBER: ~3K examples")

# ============== 5. CondAmbigQA Dataset ==============
print("\n[5/5] Downloading CondAmbigQA dataset...")
condambigqa_dir = base_dir / '05_condambigqa'
condambigqa_dir.mkdir(exist_ok=True)

condambigqa_ds = load_dataset('Apocalypse-AGI-DAO/CondAmbigQA-2K', split='train')
# Save as JSON
condambigqa_data = [ex for ex in condambigqa_ds]
with open(condambigqa_dir / 'train.json', 'w') as f:
    json.dump(condambigqa_data, f)

print("✅ CondAmbigQA: ~2K examples")

# Summary
print("\n" + "="*60)
print("✅ ALL DATASETS DOWNLOADED!")
print("="*60)
print("\n📊 Summary:")
print("   • WiC: ~6K examples")
print("   • AmbigQA: ~12K examples")
print("   • ASQA: ~5K examples")
print("   • CLAMBER: ~3K examples")
print("   • CondAmbigQA: ~2K examples")
print("   • TOTAL: ~28K raw examples → ~95K NLI examples (after conversion)")
print("="*60)

In [ ]:
# ============================================================
# CELL 4: PREPARE REAL NLI DATASET FROM 5 DATASETS
# ============================================================
print("\n" + "="*60)
print("📊 PREPARING REAL NLI DATASET FROM 5 AMBIGUITY DATASETS")
print("="*60)
print("\n⏱️  This will take 5-10 minutes...\n")

import os
import json
from pathlib import Path
import sys
sys.path.insert(0, '/content/rag-project')

# Import converters
from src.training.data.converters.ambigqa_converter import AmbigQAConverter
from src.training.data.converters.asqa_converter import ASQAConverter
from src.training.data.converters.wic_converter import WiCConverter
from src.training.data.converters.clamber_converter import CLAMBERConverter
from src.training.data.converters.condambigqa_converter import CondAmbigQAConverter

# Create output directory
os.makedirs('/content/nli_dataset', exist_ok=True)

# Dataset paths (after git clone)
base_path = Path('/content/rag-project/data/ambiguity_datasets')

# Initialize converters with actual dataset paths
converters = [
    ('AmbigQA', AmbigQAConverter(str(base_path / '02_ambigqa'), multiplier=1)),
    ('ASQA', ASQAConverter(str(base_path / '03_asqa'), multiplier=1)),
    ('WiC', WiCConverter(str(base_path / '01_wic'), multiplier=1)),
    ('CLAMBER', CLAMBERConverter(str(base_path / '04_clamber'), multiplier=1)),
    ('CondAmbigQA', CondAmbigQAConverter(str(base_path / '05_condambigqa'), multiplier=1))
]

all_examples = []
dataset_stats = {}

# Convert each dataset
for name, converter in converters:
    print(f"\n🔄 Converting {name}...")
    try:
        examples = converter.convert()
        all_examples.extend(examples)
        dataset_stats[name] = len(examples)
        print(f"   ✅ {len(examples):,} examples from {name}")
    except Exception as e:
        print(f"   ⚠️  Error with {name}: {e}")
        import traceback
        traceback.print_exc()

# ========== LIMIT DATASET SIZE ==========
# Converter'lar çok örnek üretiyor, 30K ile sınırla
import random
random.seed(42)

MAX_EXAMPLES = 30000
if len(all_examples) > MAX_EXAMPLES:
    print(f"\n⚠️  Dataset too large ({len(all_examples):,}), sampling {MAX_EXAMPLES:,} examples...")
    all_examples = random.sample(all_examples, MAX_EXAMPLES)
    print(f"✅ Sampled to {len(all_examples):,} examples")

# Save to JSONL (train file)
output_file = '/content/nli_dataset/train.jsonl'
print(f"\n💾 Saving to {output_file}...")
with open(output_file, 'w', encoding='utf-8') as f:
    for ex in all_examples:
        # Remove metadata to reduce file size
        clean_ex = {
            'premise': ex['premise'],
            'hypothesis': ex['hypothesis'],
            'label': ex['label']
        }
        f.write(json.dumps(clean_ex, ensure_ascii=False) + '\n')

# Create minimal val file (we'll use val_split=0 in trainer)
val_file = '/content/nli_dataset/val.jsonl'
with open(val_file, 'w', encoding='utf-8') as f:
    # Use first 100 examples for validation
    for ex in all_examples[:100]:
        clean_ex = {
            'premise': ex['premise'],
            'hypothesis': ex['hypothesis'],
            'label': ex['label']
        }
        f.write(json.dumps(clean_ex, ensure_ascii=False) + '\n')

# Print statistics
print("\n" + "="*60)
print("✅ DATASET READY!")
print("="*60)
print(f"\n📊 Statistics:")
print(f"   • Total examples: {len(all_examples):,}")
for dataset_name, count in dataset_stats.items():
    percentage = (count / len(all_examples)) * 100
    print(f"   • {dataset_name}: {count:,} ({percentage:.1f}%)")

# Label distribution
from collections import Counter
label_counts = Counter(ex['label'] for ex in all_examples)
label_names = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
print(f"\n📈 Label Distribution:")
for label, count in sorted(label_counts.items()):
    percentage = (count / len(all_examples)) * 100
    print(f"   • {label_names[label]}: {count:,} ({percentage:.1f}%)")

print(f"\n📁 Files:")
print(f"   • Train: {output_file}")
print(f"   • Val: {val_file}")
print("="*60)

In [ ]:
# ============================================================
# CELL 5: CONFIG (DeBERTa-BASE OPTIMIZED)
# ============================================================
print("\n" + "="*60)
print("⚙️  CONFIGURATION - DeBERTa-BASE OPTIMIZED")
print("="*60)

import yaml
import json
import os

# Create DeBERTa-BASE config
config = {
    'model': {
        'base_model': 'microsoft/deberta-v3-base',
        'num_labels': 3
    },
    'hyperparameters': {
        'num_epochs': 3,
        'batch_size': 16,             # Larger batch for smaller model
        'learning_rate': 2e-5,
        'gradient_accumulation_steps': 4,  # 32 × 2 = 64 effective
        'warmup_ratio': 0.1,
        'weight_decay': 0.01,
        'max_grad_norm': 1.0,
        'mixed_precision': 'fp16'
    },
    'data': {
        'max_seq_length': 512,        # Full sequence length
        'dataset_names': ['ambigqa', 'asqa', 'wic', 'clamber', 'condambigqa'],
        'balance_classes': True
    },
    'checkpoint': {
        'save_dir': '/content/checkpoints',
        'save_every_n_batches': 200,
        'save_total_limit': 2
    }
}

# Save config
os.makedirs('/content/rag-project/config', exist_ok=True)
with open('/content/rag-project/config/hallucination_training.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Config created (DeBERTa-BASE OPTIMIZED)")
print(f"   • Model: {config['model']['base_model']}")
print(f"   • Batch size: {config['hyperparameters']['batch_size']}")
print(f"   • Gradient accumulation: {config['hyperparameters']['gradient_accumulation_steps']}")
print(f"   • Effective batch: {config['hyperparameters']['batch_size'] * config['hyperparameters']['gradient_accumulation_steps']}")
print(f"   • Epochs: {config['hyperparameters']['num_epochs']}")
print(f"   • Max sequence length: {config['data']['max_seq_length']}")
print(f"   • Mixed precision: {config['hyperparameters']['mixed_precision']}")
print(f"\n⚡ EXPECTED PERFORMANCE:")
print(f"   • Speed: 2-3 it/s")
print(f"   • Time: ~3-4 hours (T4)")
print(f"   • Memory: ~10-15GB")
print(f"   • Model size: 560MB (vs 1.6GB)")

In [ ]:
# ============================================================
# CELL 6: CHECK & RE-CREATE CACHE IF NEEDED
# ============================================================
print("\n" + "="*60)
print("🔍 CHECKING & CREATING CACHE")
print("="*60)

import os
from pathlib import Path

# Check cache files
cache_dir = Path('/content/nli_dataset_cache')
train_cache = cache_dir / 'train.pt'
val_cache = cache_dir / 'val.pt'

if train_cache.exists() and val_cache.exists():
    print("✅ Cache files exist!")
    print(f"   • Train cache: {train_cache.stat().st_size / 1024**3:.1f}GB")
    print(f"   • Val cache: {val_cache.stat().st_size / 1024**2:.1f}MB")
    
else:
    print("❌ Cache files missing! Creating now...")
    
    # Load raw data and create cache
    import torch
    import json
    from transformers import AutoTokenizer
    from tqdm import tqdm
    
    print("📂 Loading raw data...")
    with open('/content/nli_dataset/train.jsonl', 'r') as f:
        train_data = [json.loads(line) for line in f]
    
    with open('/content/nli_dataset/val.jsonl', 'r') as f:
        val_data = [json.loads(line) for line in f]
    
    print(f"   • Train examples: {len(train_data):,}")
    print(f"   • Val examples: {len(val_data):,}")
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
    
    def tokenize_batch(examples, max_length=512):
        premises = [ex['premise'] for ex in examples]
        hypotheses = [ex['hypothesis'] for ex in examples]
        labels = [ex['label'] for ex in examples]
        
        encodings = tokenizer(
            premises, hypotheses,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'labels': torch.tensor(labels, dtype=torch.long)
        }
    
    # Tokenize train data
    print("\n🚀 Tokenizing train data (this will take 3-5 minutes)...")
    batch_size = 1000
    train_tokenized = []
    
    for i in tqdm(range(0, len(train_data), batch_size), desc="Tokenizing train"):
        batch = train_data[i:i+batch_size]
        tokenized = tokenize_batch(batch)
        train_tokenized.append(tokenized)
    
    # Combine batches
    print("🔗 Combining batches...")
    train_input_ids = torch.cat([t['input_ids'] for t in train_tokenized], dim=0)
    train_attention_mask = torch.cat([t['attention_mask'] for t in train_tokenized], dim=0)
    train_labels = torch.cat([t['labels'] for t in train_tokenized], dim=0)
    
    # Tokenize validation data
    print("\n🔧 Tokenizing validation data...")
    val_tokenized = tokenize_batch(val_data)
    
    # Save cache
    print("\n💾 Saving to cache...")
    cache_dir.mkdir(exist_ok=True)
    
    torch.save({
        'input_ids': train_input_ids,
        'attention_mask': train_attention_mask,
        'labels': train_labels
    }, train_cache)
    
    torch.save(val_tokenized, val_cache)
    
    print(f"\n✅ Cache created!")
    print(f"   • Train cache: {train_input_ids.shape}")
    print(f"   • Val cache: {val_tokenized['input_ids'].shape}")
    print(f"   • Cache size: {train_cache.stat().st_size / 1024**3:.1f}GB")

print("✅ Cache ready for training!")
print("="*60)

In [ ]:
# ============================================================
# CELL 7: FINAL TRAINING - A100 OPTIMIZED WITH CACHE
# ============================================================
print("\n" + "="*60)
print("🔥 FINAL TRAINING - DeBERTa-BASE OPTIMIZED")
print("="*60)

import yaml
from src.training.trainers.hallucination_trainer import HallucinationTrainer
import torch
from torch.utils.data import Dataset, DataLoader

# Cache'den okuyan optimize DataLoader
class CachedNLIDataset(Dataset):
    """Dataset that loads pre-tokenized data from cache."""
    
    def __init__(self, cache_file):
        self.data = torch.load(cache_file)
        self.length = self.data['input_ids'].shape[0]
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.data['input_ids'][idx],
            'attention_mask': self.data['attention_mask'][idx],
            'labels': self.data['labels'][idx]
        }

def cached_collate_fn(batch):
    """Fast collate function for cached data."""
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'labels': torch.stack([item['labels'] for item in batch])
    }

def create_cached_dataloader(cache_file, batch_size=16, shuffle=True):
    """Create OPTIMIZED DataLoader from cache (A100 Colab tuned)."""
    dataset = CachedNLIDataset(cache_file)
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=4,              # ⚡ 8 → 4 (optimal for A100 Colab)
        pin_memory=True,
        prefetch_factor=4,          # ⚡ 2 → 4 (aggressive prefetching)
        persistent_workers=True,
        drop_last=True,             # ⚡ NEW: Drop incomplete last batch
        collate_fn=cached_collate_fn
    )

# Load config directly
with open('/content/rag-project/config/hallucination_training.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 🔍 BATCH SIZE VERIFICATION
print(f"\n🔍 CONFIGURATION VERIFICATION:")
print(f"   • Batch size: {config['hyperparameters']['batch_size']}")
print(f"   • Gradient accumulation: {config['hyperparameters']['gradient_accumulation_steps']}")
print(f"   • Effective batch size: {config['hyperparameters']['batch_size'] * config['hyperparameters']['gradient_accumulation_steps']}")
print(f"   • Mixed precision: {config['hyperparameters']['mixed_precision']}")
print(f"   • Max seq length: {config['data']['max_seq_length']}")

# Initialize trainer
trainer = HallucinationTrainer(config)
print("\n✅ Trainer initialized")

# Build model
trainer.build_model()
print("✅ Model built")

# Patch trainer to use cached data
print("\n🔧 Patching trainer to use OPTIMIZED cached dataloaders...")
original_prepare_data = trainer.prepare_data

def cached_prepare_data(train_path, val_path):
    """Override prepare_data to use OPTIMIZED cached data."""
    print("🚀 Using pre-tokenized cached data with A100 optimizations!")
    
    trainer.train_loader = create_cached_dataloader(
        '/content/nli_dataset_cache/train.pt',
        batch_size=16,
        shuffle=True
    )
    trainer.val_loader = create_cached_dataloader(
        '/content/nli_dataset_cache/val.pt',
        batch_size=16,
        shuffle=False
    )
    
    # 📊 DATALOADER INFO
    print(f"\n📊 DATALOADER INFO:")
    print(f"   • Train batches/epoch: {len(trainer.train_loader):,}")
    print(f"   • Val batches: {len(trainer.val_loader):,}")
    print(f"   • Total batches (3 epochs): {len(trainer.train_loader) * 3:,}")
    print(f"   • Optimizer steps (grad_accum=4): {len(trainer.train_loader) * 5 // 4:,}")
    print(f"\n⚡ OPTIMIZATIONS:")
    print(f"   • num_workers: 4 (Colab A100 optimal)")
    print(f"   • prefetch_factor: 4 (aggressive)")
    print(f"   • drop_last: True (skip incomplete batch)")

# Apply the patch
trainer.prepare_data = cached_prepare_data

# Prepare data (will use cached data!)
trainer.prepare_data('/content/nli_dataset/train.jsonl', '/content/nli_dataset/val.jsonl')
print("\n✅ Cached data prepared")

# START TRAINING
print("\n" + "="*60)
print("🚀 TRAINING STARTING...")
print("="*60)
print(f"\n⚡ Expected Performance:")
print(f"   • Speed: 2-3 it/s (faster with smaller model!)")
print(f"   • Time: ~3-4 hours (T4)")
print(f"   • Memory: ~8-10GB")
print(f"\n📊 Progress bar shows OPTIMIZER STEPS (not batches!)")
print("\n" + "="*60 + "\n")

history = trainer.train(
    num_epochs=3,
    output_dir='/content/output'
)

In [ ]:
# ============================================================
# CELL 8: SAVE TO GOOGLE DRIVE
# ============================================================
print("\n" + "="*60)
print("💾 SAVING MODEL TO GOOGLE DRIVE")
print("="*60)

import os
import json

# Model'i kaydet
save_path = '/content/drive/MyDrive/hallucination_model_deberta_base'
os.makedirs(save_path, exist_ok=True)

print("\n[1/3] Saving model...")
trainer.model.save_pretrained(save_path)
print("✅ Model saved")

print("\n[2/3] Saving tokenizer...")
trainer.tokenizer.save_pretrained(save_path)
print("✅ Tokenizer saved")

print("\n[3/3] Saving metrics...")
with open(f'{save_path}/final_metrics.json', 'w') as f:
    json.dump({
        'validation_metrics': history.get('val_metrics', [])[-1] if history.get('val_metrics') else {},
        'training_config': {
            'num_epochs': 3,
            'batch_size': 32,
            'gradient_accumulation_steps': 2,
            'effective_batch_size': 64,
            'learning_rate': 2e-5,
            'max_seq_length': 512,
            'model': 'microsoft/deberta-v3-base',
            'optimization': 'DeBERTa-BASE - 86M params, batch=32, 3 epochs, ~28K examples'
        }
    }, f, indent=2)
print("✅ Metrics saved")

print("\n" + "="*60)
print("🎉 TAMAMLANDI!")
print("="*60)
print(f"\n📁 Model konumu:")
print(f"   Google Drive → MyDrive → hallucination_model_deberta_base/")
print(f"\n📊 Final Metrics:")
if history.get('val_metrics'):
    for key, value in history['val_metrics'][-1].items():
        if 'accuracy' in key or 'f1' in key:
            print(f"   • {key}: {value:.4f}")
print(f"\n⚡ DeBERTa-BASE TRAINING SUMMARY:")
print("   • Model: microsoft/deberta-v3-base (86M params)")
print("   • Dataset: ~28K examples (multiplier=1)")
print("   • Epochs: 3")
print("   • Batch size: 32 (effective: 64)")
print("\n✅ Model Google Drive'da güvende!")
print("="*60)